# Exercise 03: Dash/Plotly

Requirements: Numpy, pandas, dash and jupyter_dash

In [2]:

# In case you have problems, try using this specific versions: 
# !pip install -q jupyter_dash==0.3.0
# !pip install -q dash==1.19.0
# import dash_core_components as dcc
# import dash_html_components as html
!pip install jupyter_dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.9 MB 7.3 MB/s 
     |████████████████████████████████| 1.6 MB 52.2 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11448 sha256=164878b912201349384150b203eac721f4a921924f33172fb977500568ffb9c6
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


In [3]:
# We need to use a package to be able to run dash embedded in a jupyter notebook:
import jupyter_dash
jupyter_dash.__version__

'0.4.2'

In [4]:
import pandas as pd
import numpy as np
# Let's call some modules to plot and build a web app:
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash import  dcc, html  # Dash Core Components and HTML components
from dash.dependencies import Input, Output  ### Callbacks!

[Dash Core Components](https://dash.plotly.com/dash-core-components): Gives us acces to interactive components, including dropdowns, checklists, radio buttons sliders, etc.

[HTML Components](https://dash.plotly.com/dash-html-components): To create the HTML Layout using Python and not raw HTML or templates. 


Let's make a random Bar plot using graph_objects 

In [ ]:
# If you have problems running the next line, try using: 
# !pip install --upgrade nbformat

In [5]:
mu, sigma = 1, 0.5 # mean and standard deviation
s = np.random.normal(mu, sigma, 1000)
hist, bin_edges = np.histogram(s, 20, density=True)

fig = go.Figure(
    data=[go.Bar(y=hist, x = bin_edges)],
    layout_title_text="Example of a Figure"
)
fig.show()

# Preparing our data

1. Create a function to calculate the seven day average from a dataset. 

    * It takes two arguments: the datapoints and a window size. 

    * It should shift the window over the given points and calculate the mean within that window. 

In [6]:
def movingAverage(x, window=7):
    r = np.zeros_like(x)
    
    for i in range(x.shape[0]):
        if i < window:
            r[i] = x[:i+1].mean()
            
        else:        
            r[i] = x[i-window:i].mean()
        
    return r

2. Read the csv using pandas

In [9]:
df = pd.read_csv ('covid_19.csv',sep=',') # https://github.com/robert-koch-institut/SARS-CoV-2-Nowcasting_und_-R-Schaetzung
df.head()

,Datum,PS_COVID_Faelle,UG_PI_COVID_Faelle,OG_PI_COVID_Faelle,PS_COVID_Faelle_ma4,UG_PI_COVID_Faelle_ma4,OG_PI_COVID_Faelle_ma4,PS_7_Tage_R_Wert,UG_PI_7_Tage_R_Wert,OG_PI_7_Tage_R_Wert
0,2020-03-02,304,292,319,225,213,238,NaN,NaN,NaN
1,2020-03-03,321,304,337,261,248,276,NaN,NaN,NaN
2,2020-03-04,448,430,467,326,311,342,NaN,NaN,NaN
3,2020-03-05,503,485,525,394,378,412,NaN,NaN,NaN
4,2020-03-06,757,732,783,507,488,528,2.34,2.28,2.39


3. Check the data we have available:

In [8]:
df.columns

Index(['Datum', 'PS_COVID_Faelle', 'UG_PI_COVID_Faelle', 'OG_PI_COVID_Faelle',
       'PS_COVID_Faelle_ma4', 'UG_PI_COVID_Faelle_ma4',
       'OG_PI_COVID_Faelle_ma4', 'PS_7_Tage_R_Wert', 'UG_PI_7_Tage_R_Wert',
       'OG_PI_7_Tage_R_Wert'],
      dtype='object')

# Creating a Dash Layout

In [10]:
app = JupyterDash(__name__) ## Jupyter Dash app

4. Create a function that returns a bar plot using graph_objects: 

In [13]:
# Function that creates a Bar plot from  a 'df' filtering 'data' with an 'initial_date'
def plot_bar(df,colum, initial_date): 
  return go.Bar(x=df.query("Datum >'{}'".format(initial_date))['Datum'],
                y=df.query("Datum >'{}'".format(initial_date))[colum].values,
                name="raw daily cases")

5. Create a function that returns a line plot using graph_objects.

How do we ask for specific colums in our pandas data frame using a boolean equation?

In [14]:
# Function that creates a Line plot 
# It calculates the seven day average from  a 'df' filtering 'data' with an 'initial_date'
def plot_line(df, colum, initial_date):
  sevendays = movingAverage(df.query("Datum > '2021-01-01'")[colum].values)
  return go.Scatter(y = sevendays, x=df.query("Datum >'{}'".format(initial_date))['Datum'], name =' 7 day average' )

In [11]:
df.head()

,Datum,PS_COVID_Faelle,UG_PI_COVID_Faelle,OG_PI_COVID_Faelle,PS_COVID_Faelle_ma4,UG_PI_COVID_Faelle_ma4,OG_PI_COVID_Faelle_ma4,PS_7_Tage_R_Wert,UG_PI_7_Tage_R_Wert,OG_PI_7_Tage_R_Wert
0,2020-03-02,304,292,319,225,213,238,NaN,NaN,NaN
1,2020-03-03,321,304,337,261,248,276,NaN,NaN,NaN
2,2020-03-04,448,430,467,326,311,342,NaN,NaN,NaN
3,2020-03-05,503,485,525,394,378,412,NaN,NaN,NaN
4,2020-03-06,757,732,783,507,488,528,2.34,2.28,2.39


In [15]:
colum = 'PS_COVID_Faelle'      # first data to display

initial_date = '2021-01-01'    

bar = plot_bar(df,colum, initial_date)   # creating the go.Bar object

line = plot_line(df, colum,initial_date) # creating the go.Scatter object 

We can create a Figure that contains the plots:

In [16]:
 # Function that creates a Plotly Figure. 
 # The Figure displays the plots contained in 'input'.
 # The figure will have a width of 'w' and a height of 'h'
def create_fig(input, w = 900, h = 600 ):
  fig = go.Figure(data = input).update_layout(
                  width=w,  # setting  width
                  height=h, # setting  height   
                  updatemenus = list([dict(
                                type="buttons", # Buttons to activate/deactivate plots
                                active=0,       # First button will be active by default                 
                                buttons=list([
                                            # Button 1
                                            dict(label = 'Raw daily cases', # name of the button
                                                    method = 'update',        # method used to refresh the image
                                                    args = [{'visible': [True, False]}, # which plot will be available (only first) 
                                                            {'title': 'Covid-19 Cases in Germany'}]), # name of the plot
                                            # Button 2
                                            dict(label = '7 Day average', # name of the button
                                                    method = 'update',
                                                    args = [{'visible': [False, True]},
                                                            {'title': 'Covid-19 Cases in Germany'}]), 
                                            # Button 3
                                            dict(label = 'Both',
                                                    method = 'update',
                                                    args = [{'visible': [True, True]},
                                                            {'title': 'Covid-19 Cases in Germany'}])
                            ]),
                        )
                    ])
                )
  return fig


In [17]:
input = [bar, line] #bar plot, line plot

fig = create_fig(input)

6. Create a Dropdown that will update the data of your plots.

In [19]:
# Dropdown from dash_core_components
dd = dcc.Dropdown(
        id='dd',
        # options: dictionary containing all the options.
        # 'label' is the label of the option display to the user 
        # 'value' is the internal value to work with
        options=[
            {'label': 'PS', 'value': 'PS_COVID_Faelle' },       # option 1
            {'label': 'UG_PI', 'value': 'UG_PI_COVID_Faelle'},  # option 2
            {'label': 'OG_PI', 'value': 'OG_PI_COVID_Faelle'}], # option 3
            value = 'PS_COVID_Faelle' )#initial value

We create the final layout with a the ``Content Division`` element of the html:

In [20]:
app.layout = html.Div([
    html.H1("Covid cases in Germany "),   # Title
    dcc.Graph(id='graph', figure = fig),  # Graph that contains the figure (Look at the ID!!) 
    dd, # Dropdown under the figure
])

Working with callbacks

7. Create the ``callback`` to update your plot. Calcualte again the seven day average and make the plots using the new entry. Then, feed the graph with your new figure.

In [21]:
@app.callback(
    Output('graph', 'figure'),
    Input('dd', 'value')
  )

def update_output(value):
  # we re-call the function but now with a given new data obtained from the dropdown selection

  bar = plot_bar(df,value, initial_date)

  line = plot_line(df, value,initial_date)
   
  # we pass the bar plot and the line to the figure and return it to the graph.
  return create_fig([bar, line]) 

Finally, we run the app using the port 8090. You can either follow the link, or use the ``inline mode`` to display the app on a cell of Jupyter Notebook/ Google Colab.

In [22]:
if __name__ == '__main__':
  app.run_server(port = 8090, dev_tools_ui=True, debug=True,
              dev_tools_hot_reload =True, threaded=True)

Dash app running on:


<IPython.core.display.Javascript object>